In [1]:
import pandas as pd
import numpy as np
from io import StringIO

In [74]:
#Indian Corpus
count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
comb=''
counter=0
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE INDIA\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<[^<>]*>', value='', regex=True)
        temp_df = temp_df.apply(lambda temp_df: temp_df.str.lower())
        #comb = comb.append(temp_df)
        new_filepath = "..\CleanCorpora\India\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False, header=False)
        temp_df = temp_df.apply(lambda x: ' '.join(x.astype(str)), axis=1)
        temp_df = temp_df.replace("", np.nan)
        temp_df = temp_df.replace("\s+", " ", regex=True).dropna()
        comb = ' '.join(temp_df)
        comb_filepath = "..\CleanCorpora\India\_IndiaCorporaCombined{0}.txt".format(counter)
        with open(comb_filepath, 'w',encoding='UTF-8') as f:
            f.write(comb)
        counter+=1
#combDF = pd.DataFrame(eval(comb))


#combDF.to_csv(comb_filepath, sep='\t', index=False)

In [106]:
dfs = []
for i in range(300):
    #df = pd.read_csv("..\CleanCorpora\India\_IndianCorporaCombined{0}.txt".format(i), sep="\t", engine="python", encoding="UTF-8")
    with open("..\CleanCorpora\India\_IndiaCorporaCombined{0}.txt".format(i), "r", encoding='UTF-8') as file:
        text = file.read()
    df = pd.DataFrame({"Label": "India", "Document": [text]})
    dfs.append(df)
    #combdf = pd.concat([combdf, df], ignore_index=True)
comb = pd.concat(dfs, ignore_index=True)
print(comb.head())
comb.to_csv("..\CleanCorpora\India\__FULLIndiaCorporaCombined.txt", sep='\t', index=False, header=False)
India=comb
    #with open("..\CleanCorpora\_IndianCorporaCombined{0}.txt".format(i), "r", encoding='UTF-8') as file:
        #text = file.read()

   Label                                           Document
0  India  may i know your good name my name is mrs lalit...
1  India  uh okay let us continue yeah i am uh subramani...
2  India  hello we'll arrange a party  yes on the fifth ...
3  India  uhm hello ashokji i'm nice here because i'm  h...
4  India  uh hello basu  how are you uhm hello  chakrabo...


In [61]:
#HongKong Corpus
count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
comb=''
counter=0
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE-HK\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<[^<>]*>', value='', regex=True)
        temp_df = temp_df.apply(lambda temp_df: temp_df.str.lower())
        #comb = comb.append(temp_df)
        new_filepath = "..\CleanCorpora\HongKong\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False, header=False)
        temp_df = temp_df.apply(lambda x: ' '.join(x.astype(str)), axis=1)
        temp_df = temp_df.replace("", np.nan)
        temp_df = temp_df.replace("\s+", " ", regex=True).dropna()
        comb = ' '.join(temp_df)
        comb_filepath = "..\CleanCorpora\HongKong\_HongKongCorporaCombined{0}.txt".format(counter)
        with open(comb_filepath, 'w',encoding='UTF-8') as f:
            f.write(comb)
        counter+=1
#combDF = pd.DataFrame(eval(comb))


#combDF.to_csv(comb_filepath, sep='\t', index=False)

In [107]:
dfs = []
for i in range(300):
    with open("..\CleanCorpora\HongKong\_HongKongCorporaCombined{0}.txt".format(i), "r", encoding='UTF-8') as file:
        text = file.read()
    df = pd.DataFrame({"Label": "HongKong", "Document": [text]})
    dfs.append(df)
    #combdf = pd.concat([combdf, df], ignore_index=True)
comb = pd.concat(dfs, ignore_index=True)
print(comb.head())
comb.to_csv("..\CleanCorpora\HongKong\__FULLHongKongCorporaCombined.txt", sep='\t', index=False, header=False)
HongKong=comb

      Label                                           Document
0  HongKong  yeah   first of all uhm this isn't the best wa...
1  HongKong  uhm so you want to go to beijing this summer u...
2  HongKong  uhm what about you can you introduce yourself ...
3  HongKong  uhm anything you ' re particularly interested ...
4  HongKong  uh no no no no i i i think uhm even i don't at...


In [66]:
#Philippines Corpus
count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
comb=''
counter=0
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE Philippines\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<[^<>]*>', value='', regex=True)
        temp_df = temp_df.apply(lambda temp_df: temp_df.str.lower())
        #comb = comb.append(temp_df)
        new_filepath = "..\CleanCorpora\Philippines\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False, header=False)
        temp_df = temp_df.apply(lambda x: ' '.join(x.astype(str)), axis=1)
        temp_df = temp_df.replace("", np.nan)
        temp_df = temp_df.replace("\s+", " ", regex=True).dropna()
        comb = ' '.join(temp_df)
        comb_filepath = "..\CleanCorpora\Philippines\_PhilippinesCorporaCombined{0}.txt".format(counter)
        with open(comb_filepath, 'w',encoding='UTF-8') as f:
            f.write(comb)
        counter+=1
#combDF = pd.DataFrame(eval(comb))


#combDF.to_csv(comb_filepath, sep='\t', index=False)

In [112]:
dfs = []
for i in range(300):
    with open("..\CleanCorpora\Philippines\_PhilippinesCorporaCombined{0}.txt".format(i), "r", encoding='UTF-8') as file:
        text = file.read()
    df = pd.DataFrame({"Label": "Philippines", "Document": [text]})
    dfs.append(df)
    #combdf = pd.concat([combdf, df], ignore_index=True)
comb = pd.concat(dfs, ignore_index=True)
print(comb.head())
comb.to_csv("..\CleanCorpora\Philippines\__FULLPhilippinesCorporaCombined.txt", sep='\t', index=False, header=False)
Philippines=comb

         Label                                           Document
0  Philippines  oh so how are you how 's life uh it 's fine ra...
1  Philippines  i 'm leaving next next next monday it 's on th...
2  Philippines  how many books are you writing sir brother and...
3  Philippines  you 're in white you 're so pure and virginal ...
4  Philippines  so we 'd like to have a brief account of your ...


In [69]:
#Singapore Corpus
count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
comb=''
counter=0
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE Singapore\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<[^<>]*>', value='', regex=True)
        temp_df = temp_df.apply(lambda temp_df: temp_df.str.lower())
        #comb = comb.append(temp_df)
        new_filepath = "..\CleanCorpora\Singapore\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False, header=False)
        temp_df = temp_df.apply(lambda x: ' '.join(x.astype(str)), axis=1)
        temp_df = temp_df.replace("", np.nan)
        temp_df = temp_df.replace("\s+", " ", regex=True).dropna()
        comb = ' '.join(temp_df)
        comb_filepath = "..\CleanCorpora\Singapore\_SingaporeCorporaCombined{0}.txt".format(counter)
        with open(comb_filepath, 'w',encoding='UTF-8') as f:
            f.write(comb)
        counter+=1
#combDF = pd.DataFrame(eval(comb))


#combDF.to_csv(comb_filepath, sep='\t', index=False)

In [109]:
dfs = []
for i in range(300):
    with open("..\CleanCorpora\Singapore\_SingaporeCorporaCombined{0}.txt".format(i), "r", encoding='UTF-8') as file:
        text = file.read()
    df = pd.DataFrame({"Label": "Singapore", "Document": [text]})
    dfs.append(df)
    #combdf = pd.concat([combdf, df], ignore_index=True)
comb = pd.concat(dfs, ignore_index=True)
print(comb.head())
comb.to_csv("..\CleanCorpora\Singapore\__FULLSingaporeCorporaCombined.txt", sep='\t', index=False, header=False)
Singapore=comb

       Label                                           Document
0  Singapore  is that that's not enough for you ah thirsty i...
1  Singapore  on her own uhm i think so because she's stayin...
2  Singapore  i believe i just had one okay how about yours ...
3  Singapore  why did they revert back to the end of year ex...
4  Singapore   29 december 1991  feel like counting used to ...


In [116]:
#Concatenate all 4 dialects
dfs = pd.concat([India, HongKong, Philippines, Singapore], ignore_index=True)
dfs.shape
dfs.to_csv("..\CleanCorpora\Combined\Combined.txt", sep='\t', index=False, header=False)

In [124]:
#Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [129]:
tfidf = TfidfVectorizer(stop_words='english')
dfs_x = dfs["Document"]
dfs_y = dfs["Label"]
x_train, x_test, y_train, y_test = train_test_split(dfs_x, dfs_y, test_size=0.2, random_state=4)
x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)
clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 93.75%
